<a href="https://colab.research.google.com/github/muhammetsnts/SPARK/blob/main/projects/4.Customer_Churn_with_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Info

A marketing agency has many customers that use their service to produce ads for the clients/customer websites. They've noticed that they have quite a bit of churn in clients.

The want to predict which customer will churn, so that they can correctly assign the customers most at risk to churn an account manager.

We will create a classification algorithm by using `customer_churn.csv` dataset.Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    

NOTE: In dataset, Account Manager is assigned randomly to the customers!!!


# Setting Environment

In [1]:
# install Java8
!apt-get -q install openjdk-8-jdk-headless -qq > /dev/null

# download spark3.1.1
!wget -q https://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

# unzip it
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

# install findspark 
!pip install -q findspark


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"


import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Download and Read the Data

In [2]:
!wget -q https://raw.githubusercontent.com/muhammetsnts/SPARK/main/data/customer_churn.csv

# Create Spark DataFrame

In [3]:
data = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

# Data Cleaning

- There is no missing value.
- Names of the customers won't be helpful for us. 
- Account Manager is assigned randomly, so we don't expect that it affects the result but, we will use this.
- So, lets check the company column in data.

In [10]:
data.groupBy('Company').count().show()

+--------------------+-----+
|             Company|count|
+--------------------+-----+
|Miller, Johnson a...|    1|
|Hunter, Reyes and...|    1|
|          Obrien PLC|    1|
|            Soto PLC|    2|
|            Todd LLC|    1|
|Smith, Marshall a...|    1|
|           Smith PLC|    1|
|          Hall Group|    1|
|Freeman, Lam and ...|    1|
|       Smith-Carroll|    1|
|Hall, Hernandez a...|    1|
|          Cannon Inc|    1|
|        White-Dennis|    1|
|Wilson, Collins a...|    1|
|Jennings, Gates a...|    1|
|     Campbell-Willis|    1|
|    Martinez-Roberts|    1|
|        Robinson PLC|    1|
|          Barton Inc|    1|
|Hernandez, Middle...|    1|
+--------------------+-----+
only showing top 20 rows



In [13]:
data.select('Company').distinct().count()

873

As you can see, there are 873 distict companies in Conpany column. So making one-hot encoding to them will be inaccurate.

Lets take a look at the Location column.

In [16]:
data.limit(3).select('Location').show(truncate=False)

+--------------------------------------------------+
|Location                                          |
+--------------------------------------------------+
|10265 Elizabeth Mission Barkerburgh, AK 89518     |
|6157 Frank Gardens Suite 019 Carloshaven, RI 17756|
|1331 Keith Court Alyssahaven, DE 90114            |
+--------------------------------------------------+



We can get the zipcodes and states from this column and make a deeper analysis but, in this example we won't do that.

## Correlation Check

In [ ]:
from pyspark.sql.functions import corr

In [ ]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [ ]:
data.select(corr('Churn', 'Account_Manager')).show()

+----------------------------+
|corr(Churn, Account_Manager)|
+----------------------------+
|         0.07061077173214911|
+----------------------------+



As you can see the Account_Manager is not helpful for predicting Churn.

In [ ]:
data.select(corr('Churn', 'Years')).show()

+------------------+
|corr(Churn, Years)|
+------------------+
|0.2143285801724283|
+------------------+



In [ ]:
data.select(corr('Churn', 'Num_Sites')).show()

+----------------------+
|corr(Churn, Num_Sites)|
+----------------------+
|    0.5253980778988574|
+----------------------+



# Preparing the Data For Model

We will use only the numeric columns.

In [17]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [19]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [20]:
assembler = VectorAssembler(inputCols=['Age',
                                       'Total_Purchase',
                                       'Account_Manager',
                                       'Years',
                                       'Num_Sites'], 
                            outputCol='features')

We will transform our data before splitting.

In [21]:
output = assembler.transform(data)

output.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- features: vector (nullable = true)



We will choose only `Churn` and `features` columns.

In [22]:
final_data = output.select('Churn', 'features')

final_data.show()

+-----+--------------------+
|Churn|            features|
+-----+--------------------+
|    1|[42.0,11066.8,0.0...|
|    1|[41.0,11916.22,0....|
|    1|[38.0,12884.75,0....|
|    1|[42.0,8010.76,0.0...|
|    1|[37.0,9191.58,0.0...|
|    1|[48.0,10356.02,0....|
|    1|[44.0,11331.58,1....|
|    1|[32.0,9885.12,1.0...|
|    1|[43.0,14062.6,1.0...|
|    1|[40.0,8066.94,1.0...|
|    1|[30.0,11575.37,1....|
|    1|[45.0,8771.02,1.0...|
|    1|[45.0,8988.67,1.0...|
|    1|[40.0,8283.32,1.0...|
|    1|[41.0,6569.87,1.0...|
|    1|[38.0,10494.82,1....|
|    1|[45.0,8213.41,1.0...|
|    1|[43.0,11226.88,0....|
|    1|[53.0,5515.09,0.0...|
|    1|[46.0,8046.4,1.0,...|
+-----+--------------------+
only showing top 20 rows



# Train-Test Split

In [23]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [24]:
train_data.describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                631|
|   mean|0.17274167987321712|
| stddev| 0.3783236979973365|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [25]:
test_data.describe().show()

+-------+-------------------+
|summary|              Churn|
+-------+-------------------+
|  count|                269|
|   mean| 0.1524163568773234|
| stddev|0.36009394176469195|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



# Modelling

In [28]:
from pyspark.ml.classification import LogisticRegression

In [29]:
lr = LogisticRegression(labelCol='Churn')

In [30]:
lrModel = lr.fit(train_data)

# Evaluate Model

In [31]:
training_sum = lrModel.summary

In [32]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                631|                631|
|   mean|0.17274167987321712|0.13153724247226625|
| stddev| 0.3783236979973365|  0.338255113817212|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [34]:
pred_and_label = lrModel.evaluate(test_data)

In [36]:
pred_and_label.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|Churn|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|[26.0,8939.61,0.0...|[6.27004738245957...|[0.99811143450140...|       0.0|
|    0|[27.0,8628.8,1.0,...|[5.35937599457367...|[0.99531818211028...|       0.0|
|    0|[28.0,9090.43,1.0...|[1.56305181501164...|[0.82679083250873...|       0.0|
|    0|[28.0,11128.95,1....|[4.24863886242493...|[0.98591748715031...|       0.0|
|    0|[28.0,11204.23,0....|[2.0485303689569,...|[0.88579903643169...|       0.0|
|    0|[29.0,5900.78,1.0...|[3.92844982012056...|[0.98070545630293...|       0.0|
|    0|[29.0,9617.59,0.0...|[4.35637941000225...|[0.98733765444207...|       0.0|
|    0|[29.0,13255.05,1....|[4.3249535352872,...|[0.98693869052088...|       0.0|
|    0|[30.0,8403.78,1.0...|[5.88761969371315...|[0.99723410062391...|       0.0|
|    0|[30.0,867

Lets check the AUC of ROC.

In [33]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [37]:
churn_eval = BinaryClassificationEvaluator(labelCol='Churn', rawPredictionCol='prediction')

In [38]:
auc = churn_eval.evaluate(pred_and_label.predictions)

In [39]:
auc

0.720742404792469

This is not so bad, area under the curve is over 0.5.

# Model Deploying

Now we will try to predict the customer churn on new data. Before doing that, we will fit all the data to our model, not trainin or test, whole dataset.

In [51]:
final_model = lr.fit(final_data)

In [ ]:
!wget -q https://raw.githubusercontent.com/muhammetsnts/SPARK/main/data/new_customers.csv

In [52]:
new_data = spark.read.csv('new_customers.csv', header=True, inferSchema=True)
new_data.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:46|Unit 0789 Box 073...|  

In [53]:
new_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [55]:
test_new_customers = assembler.transform(new_data)

In [56]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [57]:
final_results = final_model.transform(test_new_customers)

In [59]:
final_results.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|[2.22168705251434...|[0.90218018099704...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|[-6.2207530595013...|[0.00198380445829...|       

In [60]:
final_results.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [61]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                  6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|               null|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|         